In [6]:
!nvidia-smi

Sun Nov 23 08:29:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [4]:
import os

# 查看输入目录下的内容
print("Input directory:", os.listdir('/kaggle/input'))

# 遍历 input 目录
for root, dirs, files in os.walk('/kaggle/input', topdown=True):
    print(root, dirs, files)

Input directory: ['.virtual_documents']
/kaggle/working ['.virtual_documents'] []
/kaggle/working/.virtual_documents [] []


In [2]:
import torch
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import subprocess
import psutil
import os

os.environ["WANDB_DISABLED"] = "true"


# ======================
# 内存监控函数
# ======================
def print_memory_usage(tag=""):
    print(f"\n===== Memory Usage {tag} =====")
    try:
        gpu_info = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.total", "--format=csv,noheader,nounits"]
        )
        gpu_info = gpu_info.decode("utf-8").strip().split("\n")
        for i, gpu in enumerate(gpu_info):
            used, total = map(int, gpu.split(','))
            print(f"GPU {i}: {used}MB / {total}MB")
    except:
        print("No GPU found")

    ram = psutil.virtual_memory()
    print(f"CPU RAM: {ram.used/1024**2:.2f}MB / {ram.total/1024**2:.2f}MB")
    print("====================================\n")


# ======================
# Trainer 回调类
# ======================
from transformers import TrainerCallback

class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0 and state.global_step > 0:
            print_memory_usage(f"Training step {state.global_step}")


# ==========================================================
# ============ 0. 检查是否已有 tokenized 缓存 ================
# ==========================================================
cache_path = "/tokenized_enron_spam"

if os.path.exists(cache_path):
    print("🔄 Loading cached tokenized dataset...")
    tokenized_datasets = load_from_disk(cache_path)

else:
    print("🆕 No cache found — processing raw dataset...")

    # ======================
    # 1. Load Dataset
    # ======================
    df = pd.read_csv("/kaggle/input/classification-data/enron_spam_data.csv")
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.2)

    # ======================
    # 2. Preprocess Data
    # ======================
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def prepare_dataset(example):
        example["label"] = 1 if example["Spam/Ham"].lower() == "spam" else 0
        return example

    dataset = dataset.map(prepare_dataset)

    def preprocess_function(examples):
        messages = [msg if msg is not None else "" for msg in examples["Message"]]
        return tokenizer(messages, truncation=True, padding="max_length")

    dataset = dataset.filter(lambda x: x["Message"] is not None and len(str(x["Message"]).strip()) > 0)

    tokenized_datasets = dataset.map(preprocess_function, batched=True)

    # 保存缓存
    print("💾 Saving tokenized dataset...")
    tokenized_datasets.save_to_disk(cache_path)
    print(f"✔ Tokenized dataset saved to {cache_path}")

# tokenizer 必须在外部创建一次
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Label map
id2label = {0: "HAM", 1: "SPAM"}
label2id = {"HAM": 0, "SPAM": 1}

# ======================
# 3. Load Model
# ======================
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# ======================
# 4. Metrics
# ======================
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# ======================
# 5. Training Arguments
# ======================
training_args = TrainingArguments(
    output_dir="kagglespam_classification_results",
    learning_rate=2e-5,
    dataloader_num_workers=0,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# ======================
# 6. Trainer
# ======================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()],
)

# ======================
# 7. Train
# ======================
print_memory_usage("Before training")
print("Starting training...")
trainer.train()
print_memory_usage("After training")

# ======================
# 8. Evaluate
# ======================
print("Evaluating...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# ======================
# 9. Save Model
# ======================
trainer.save_model("spam_classifier_model__full")
print("Model saved to spam_classifier_model__full")


2025-11-23 08:43:59.951971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763887440.336696      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763887440.478062      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


===== Memory Usage Training step 400 =====
GPU 0: 11411MB / 15360MB
GPU 1: 10621MB / 15360MB
CPU RAM: 3506.30MB / 32102.90MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



===== Memory Usage After training =====
GPU 0: 11411MB / 15360MB
GPU 1: 10621MB / 15360MB
CPU RAM: 3494.85MB / 32102.90MB

Evaluating...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluation results: {'eval_loss': 0.030358023941516876, 'eval_accuracy': 0.9902607132154629, 'eval_runtime': 58.0805, 'eval_samples_per_second': 114.91, 'eval_steps_per_second': 0.913, 'epoch': 2.0}
Model saved to spam_classifier_model__full


In [3]:
import os
import subprocess
import psutil
import numpy as np
import pandas as pd

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    TrainerCallback
)
from datasets import Dataset, load_from_disk
from peft import LoraConfig, get_peft_model
import evaluate  # 使用 import evaluate 而非 from evaluate import load

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ===================================
# 内存监控函数
# ===================================
def print_memory_usage(tag=""):
    print(f"\n===== Memory Usage {tag} =====")
    try:
        gpu_info = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.total", "--format=csv,noheader,nounits"]
        )
        gpu_info = gpu_info.decode("utf-8").strip().split("\n")
        for i, gpu in enumerate(gpu_info):
            used, total = map(int, gpu.split(','))
            print(f"GPU {i}: {used}MB / {total}MB")
    except Exception as e:
        print("No GPU found or nvidia-smi not available")

    ram = psutil.virtual_memory()
    print(f"CPU RAM: {ram.used/1024**2:.2f}MB / {ram.total/1024**2:.2f}MB")
    print("====================================\n")


# ===================================
# Trainer 显存监控 Callback
# ===================================
class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0 and state.global_step > 0:
            print_memory_usage(f"Training step {state.global_step}")


# ==========================================================
# ============ 0. 检查是否已有 tokenized 缓存 ================
# ==========================================================
cache_path = "/tokenized_enron_spam"

if os.path.exists(cache_path):
    print("🔄 Loading cached tokenized dataset...")
    tokenized_datasets = load_from_disk(cache_path)
else:
    print("🆕 No cache found — processing raw dataset...")

    # -----------------------
    # 1. Load raw CSV
    # -----------------------
    df = pd.read_csv("/kaggle/input/classification-data/enron_spam_data.csv")
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.2)

    # -----------------------
    # 2. Preprocess labels & text
    # -----------------------
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def prepare_dataset(example):
        example["label"] = 1 if example["Spam/Ham"].lower() == "spam" else 0
        return example

    dataset = dataset.map(prepare_dataset)

    def preprocess_function(examples):
        messages = [msg if msg is not None else "" for msg in examples["Message"]]
        return tokenizer(messages, truncation=True, padding="max_length", max_length=512)

    # Filter out empty messages
    dataset = dataset.filter(lambda x: x["Message"] is not None and len(str(x["Message"]).strip()) > 0)

    tokenized_datasets = dataset.map(preprocess_function, batched=True, batch_size=1000)

    # Save cache
    print("💾 Saving tokenized dataset...")
    tokenized_datasets.save_to_disk(cache_path)
    print(f"✔ Tokenized dataset saved to {cache_path}")

# Re-initialize tokenizer (ensure consistent instance)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

id2label = {0: "HAM", 1: "SPAM"}
label2id = {"HAM": 0, "SPAM": 1}


# ===================================
# 2. Load Model with FP16 + LoRA
# ===================================
print("Loading DistilBERT for LoRA...")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    torch_dtype=torch.float16,  # FP16 for memory efficiency
)

# ===================================
# 3. LoRA config
# ===================================
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],  # DistilBERT specific
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


# ===================================
# 4. Metrics
# ===================================
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# ===================================
# 5. Training Args
# ===================================
training_args = TrainingArguments(
    output_dir="/kaggle/working/spam_classification_LoRA_result",
    learning_rate=1e-4,
    dataloader_num_workers=0,  # Avoid tokenizers fork warning
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_grad_norm=1.0,
    report_to="none",  # Suppress WANDB warning
)


# ===================================
# 6. Trainer
# ===================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()],
)

print_memory_usage("Before LoRA training")
print("🚀 Starting LoRA training...")
trainer.train()
print_memory_usage("After LoRA training")


# ===================================
# 7. Evaluation
# ===================================
print("Evaluating...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


# ===================================
# 8. Save LoRA adapter
# ===================================
trainer.save_model("/kaggle/working/spam_classifier_LoRA_model")
print("LoRA model saved.")

2025-11-23 06:07:55.601300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763878076.023848      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763878076.145067      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


===== Memory Usage Training step 400 =====
GPU 0: 7017MB / 15360MB
GPU 1: 7015MB / 15360MB
CPU RAM: 3511.20MB / 32102.89MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3177: RuntimeWarning: invalid value encountered in less
  if operator(metric_value, self.state.best_metric):



===== Memory Usage After LoRA training =====
GPU 0: 7017MB / 15360MB
GPU 1: 7015MB / 15360MB
CPU RAM: 3501.22MB / 32102.89MB

Evaluating...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluation results: {'eval_loss': nan, 'eval_accuracy': 0.4876144723014562, 'eval_runtime': 18.5303, 'eval_samples_per_second': 359.464, 'eval_steps_per_second': 2.86, 'epoch': 2.0}
LoRA model saved.


In [3]:
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, load_from_disk

os.environ["WANDB_DISABLED"] = "true"


# =========================================================
#  显存监控函数
# =========================================================
def print_memory_usage(tag=""):
    print(f"\n===== Memory Usage {tag} =====")
    try:
        gpu_info = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.total",
             "--format=csv,noheader,nounits"]
        ).decode("utf-8").strip().split("\n")

        for i, line in enumerate(gpu_info):
            used, total = map(int, line.split(","))
            print(f"GPU {i}: {used}MB / {total}MB")
    except:
        print("No GPU detected")

    ram = psutil.virtual_memory()
    print(f"CPU RAM: {ram.used/1024**2:.2f}MB / {ram.total/1024**2:.2f}MB")
    print("======================================\n")


# =========================================================
# Callback：每 100 steps 输出一次显存
# =========================================================
class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0 and state.global_step > 0:
            print_memory_usage(f"Training step {state.global_step}")


# =========================================================
#  Tokenized Cache 目录
# =========================================================
cache_dir = "/content/drive/MyDrive/gpt2_reply_tokenized_cache"


# =========================================================
# Step 1：加载或创建 Tokenized Cache
# =========================================================
if os.path.exists(cache_dir):
    print("🔄 Loading cached tokenized dataset...")
    tokenized_datasets = load_from_disk(cache_dir)

else:
    print("⚠ No cache found! Creating tokenized dataset...")

    # 读取原始数据
    print("Loading synthetic dataset...")
    df = pd.read_csv("/kaggle/input/generation-data/synthetic_reply_dataset.csv")
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.1)

    # Tokenizer
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # 预处理函数
    def preprocess_function(examples):
        inputs = examples["Message"]
        targets = examples["reply"]

        texts = []
        for i, t in zip(inputs, targets):
            i = str(i) if i is not None else ""
            t = str(t) if t is not None else ""
            text = f"Email: {i[:512]}\n\nReply: {t}{tokenizer.eos_token}"
            texts.append(text)

        return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

    print("Tokenizing dataset (first time, this is slow)...")
    tokenized_datasets = dataset.map(preprocess_function, batched=True)

    # 保存缓存
    print(f"Saving tokenized dataset to cache: {cache_dir}")
    tokenized_datasets.save_to_disk(cache_dir)

    print("✅ Cache created!")


# =========================================================
# Step 2：加载模型和 tokenizer
# =========================================================
model_name = "gpt2"
print(f"Loading model: {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)


# =========================================================
# Step 3：TrainingArguments（模型保存到 Google Drive）
# =========================================================
output_dir = "/kaggle/working/reply_generation_model_full"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    dataloader_num_workers=0,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=f"{output_dir}/logs",
)


# =========================================================
# Step 4：构建 Trainer（含显存监控）
# =========================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[MemoryCallback()],
)


# =========================================================
# Step 5：开始训练
# =========================================================
print_memory_usage("Before training")
print("🚀 Starting training...")
trainer.train()
print_memory_usage("After training")


# =========================================================
# Step 6：保存最终模型
# =========================================================
print(f"Saving final model and tokenizer to {output_dir}...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Done! Model saved to Google Drive.")


2025-11-23 07:53:15.986000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763884396.171969      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763884396.226954      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


===== Memory Usage Training step 500 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3557.25MB / 32102.90MB


===== Memory Usage Training step 600 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3572.18MB / 32102.90MB


===== Memory Usage Training step 700 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3575.00MB / 32102.90MB


===== Memory Usage Training step 800 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3563.14MB / 32102.90MB


===== Memory Usage Training step 900 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3554.83MB / 32102.90MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



===== Memory Usage Training step 1000 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3583.08MB / 32102.90MB


===== Memory Usage Training step 1100 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3582.09MB / 32102.90MB


===== Memory Usage Training step 1200 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3593.32MB / 32102.90MB


===== Memory Usage Training step 1300 =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3571.47MB / 32102.90MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



===== Memory Usage After training =====
GPU 0: 11395MB / 15360MB
GPU 1: 3149MB / 15360MB
CPU RAM: 3572.68MB / 32102.90MB

Saving final model and tokenizer to /kaggle/working/reply_generation_model_full...
✅ Done! Model saved to Google Drive.


In [7]:
import os
import subprocess
import psutil
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, load_from_disk
from peft import LoraConfig, get_peft_model

# 禁用 W&B
os.environ["WANDB_DISABLED"] = "true"


# =========================================================
# 显存监控函数
# =========================================================
def print_memory_usage(tag=""):
    print(f"\n===== Memory Usage {tag} =====")
    try:
        gpu_info = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.total",
             "--format=csv,noheader,nounits"]
        ).decode("utf-8").strip().split("\n")

        for i, line in enumerate(gpu_info):
            used, total = map(int, line.split(","))
            print(f"GPU {i}: {used}MB / {total}MB")
    except:
        print("No GPU detected")

    ram = psutil.virtual_memory()
    print(f"CPU RAM: {ram.used/1024**2:.2f}MB / {ram.total/1024**2:.2f}MB")
    print("======================================\n")


# =========================================================
# Callback：每 100 steps 输出显存
# =========================================================
class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0 and state.global_step > 0:
            print_memory_usage(f"Training step {state.global_step}")


# =========================================================
# Tokenized Cache 目录
# =========================================================
cache_dir = "/content/drive/MyDrive/gpt2_reply_tokenized_cache_lora"


# =========================================================
# Step 1：加载或创建 Tokenized Cache
# =========================================================
if os.path.exists(cache_dir):
    print("🔄 Loading cached tokenized dataset...")
    tokenized_datasets = load_from_disk(cache_dir)
else:
    print("⚠ No cache found! Creating tokenized dataset...")

    print("Loading synthetic dataset...")
    df = pd.read_csv("/kaggle/input/generation-data/synthetic_reply_dataset.csv")
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.1)

    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    def preprocess_function(examples):
        inputs = examples["Message"]
        targets = examples["reply"]

        texts = []
        for i, t in zip(inputs, targets):
            i = str(i) if i else ""
            t = str(t) if t else ""
            text = f"Email: {i[:512]}\n\nReply: {t}{tokenizer.eos_token}"
            texts.append(text)

        return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

    print("Tokenizing dataset (this may take time)...")
    tokenized_datasets = dataset.map(preprocess_function, batched=True)

    print(f"Saving tokenized dataset to cache: {cache_dir}")
    tokenized_datasets.save_to_disk(cache_dir)
    print("✅ Cache created!")


# =========================================================
# Step 2：加载 GPT-2 + LoRA
# =========================================================
print("Loading GPT-2 + LoRA...")

base_model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    torch_dtype=torch.float16
)

# GPT-2 attention 模块名称
target_modules = ["c_attn", "c_proj"]

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=target_modules,
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


# =========================================================
# Step 3：TrainingArguments（与全参数训练一致）
# =========================================================
output_dir = "/kaggle/working/reply_generation_model_LoRA"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    dataloader_num_workers=0,                 
    per_device_train_batch_size=16,           
    per_device_eval_batch_size=16,            
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=f"{output_dir}/logs",
    fp16=True,
)


# =========================================================
# Step 4：Trainer（含显存监控）
# =========================================================
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[MemoryCallback()],
)


# =========================================================
# Step 5：训练
# =========================================================
print_memory_usage("Before LoRA Training")
print("🚀 Starting LoRA training...")
trainer.train()
print_memory_usage("After LoRA Training")


# =========================================================
# Step 6：保存 LoRA 模型
# =========================================================
print(f"Saving LoRA model to {output_dir}...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Done! LoRA model saved to Google Drive.")

🔄 Loading cached tokenized dataset...
Loading GPT-2 + LoRA...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



===== Memory Usage Before LoRA Training =====
GPU 0: 373MB / 15360MB
GPU 1: 3MB / 15360MB
CPU RAM: 2947.89MB / 32102.90MB

🚀 Starting LoRA training...


...


===== Memory Usage Training step 500 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3710.10MB / 32102.90MB


===== Memory Usage Training step 600 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3723.44MB / 32102.90MB


===== Memory Usage Training step 700 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3720.54MB / 32102.90MB


===== Memory Usage Training step 800 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3704.08MB / 32102.90MB


===== Memory Usage Training step 900 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3724.56MB / 32102.90MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



===== Memory Usage Training step 1000 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3718.64MB / 32102.90MB


===== Memory Usage Training step 1100 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3703.11MB / 32102.90MB


===== Memory Usage Training step 1200 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3700.98MB / 32102.90MB


===== Memory Usage Training step 1300 =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3699.83MB / 32102.90MB



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



===== Memory Usage After LoRA Training =====
GPU 0: 6587MB / 15360MB
GPU 1: 2821MB / 15360MB
CPU RAM: 3700.64MB / 32102.90MB

Saving LoRA model to /kaggle/working/reply_generation_model_LoRA...
✅ Done! LoRA model saved to Google Drive.
